# Signal functions in depth

`SignalFunction`s are functions that take in one or more sigals and return a new signal. They can be one of two types:

* one-to-one functions take in one sample and return one sample. They can be stateful and depend on previous samples, but the number of samples out should always be the same as number of samples in.
* many-to-one (Windowed) functions operate on a sliding window of samples. The windows can overlap, but the function only returns one value per window. They can still receive a single sample at a time but will only return a value when the window has been filled. As well as defining how to process a window they need to define the window length and window overlap. 

In [ ]:
from genki_signals.system import System
from genki_signals.sources import MicSource, CameraSource, Sampler, MouseSource
import genki_signals.functions as f

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)

A good example of a windowed signal function is short-time Fourier transform (stft) which computes the Discrete Fourier Transform on a sliding window:

In [ ]:
mic = MicSource()
stft = f.FourierTransform("audio", "fourier", window_size=1024, window_overlap=512)
fourier_system = System(mic, [stft])

fourier_system.start()

One way to view data from a `System` is to connect it to a buffer with `System.register_data_feed` which takes in some ID and some callback function. This can be useful to see if a `SignalFunction` is behaving properly

In [ ]:
from genki_signals.buffers import DataBuffer

buffer = DataBuffer()

fourier_system.register_data_feed(id(buffer), lambda d: buffer.extend(d))

Now all data that goes through our `System` will be sent to the buffer. Below we can view our buffer.

In [ ]:
buffer

In [ ]:
fourier_system.stop()

Note that in this case, the time series called `'fourier'` is shorter than `'audio'` - it is downsampled (we receive one sample of `'fourier'` per 512 samples of `'audio'`). The `DataBuffer` will not enforce synchronisation across the time dimension of signals, but with great power comes great reponsibility. This can be a source of bugs since one tends to assume that timeseries grouped together are operating on the same frequency.

Another example of a `SignalFunction` are the `Inference` and `WindowedInference` functions which take in a onnx file and calculate real-time inference with the neural network.

Let's load a model which predicts the relative depth of an image. For this example to work you need to have [pytorch](https://pytorch.org/) and [timm](https://pypi.org/project/timm/) installed.

(Note: You can also create your own `Inference` function but more on custom `SignalFunctions` later.)

We start by loading the model and creating an ONNX file:

In [ ]:
import torch

#load the model from torch.hub
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")

input_resolution = (256, 256)
dummy_input = torch.randn((1, 3, *input_resolution))
model_path = "./midas.onnx"

# export the model to onnx
torch.onnx.export(
    midas,
    dummy_input,
    model_path,
    input_names=["input"],
    output_names=["output"],
    # dynamic_axes=({"input": [0]})
)

To connect the model to our camera we need to create a `CameraSource`, a `Sampler` to sample it, and connect it to a `System` which also computes the inference

In [ ]:
camera = CameraSource(resolution=input_resolution)
camera_sampler = Sampler({"model_input": camera}, 10)
model_inference = f.Inference("model_input", "model_output", model_path, stateful=False)
inference_system = System(camera_sampler, [model_inference], update_rate=50)
inference_system.start()

The `Inference` function takes in an input name, output name, model path, and the boolean parameter `stateful`. When `stateful` is `False`, the model works on one input sample at a time independently. 

With `stateful=True`, the model is assumed to also take in a state vector as input, and output a new state vector along with the output (i.e. it is a Recurrent Neural Network), allowing it to operate with some historical context.

Now the depth image is under the key "model_output" and we can visualize it with our `WidgetFrontend`

In [ ]:
from genki_signals.frontends import Video, WidgetDashboard

video = Video(inference_system, "model_input")
depth = Video(inference_system, "model_output")

frontend = WidgetDashboard([video, depth])

frontend

In [ ]:
inference_system.stop()

Notes on using multiple signal functions:

If one `SignalFunction` depends on the output of another `SignalFunction` then it needs to come after the other in the list of functions when initializing the `System`

An example of a sequence of `SignalFunction`s:

In [ ]:
pos_to_vel = f.Differentiate("mouse_pos", "timestamp", "mouse_vel")
vel_to_acc = f.Differentiate("mouse_vel", "timestamp", "mouse_acc")
acc_to_vel = f.Integrate("mouse_acc", "timestamp", "mouse_vel_2")
vel_to_pos = f.Integrate("mouse_vel_2", "timestamp", "mouse_pos_2", use_trapz=False)

mouse_source = MouseSource()
sampler = Sampler({"mouse_pos": mouse_source}, 100)
system = System(sampler, [pos_to_vel, vel_to_acc, acc_to_vel, vel_to_pos])

system.start()

In [ ]:
from genki_signals.frontends import Line, WidgetDashboard

pos =  Line(system, "timestamp", "mouse_pos")
pos2 = Line(system, "timestamp", "mouse_pos_2")
vel =  Line(system, "timestamp", "mouse_vel")
vel2 = Line(system, "timestamp", "mouse_vel_2")

frontend = WidgetDashboard([pos, pos2, vel, vel2])
frontend

In [ ]:
system.stop()

Note that the "position" we end up with after differentiating twice and itegrating twice again is a very poor approximation, and drifts a lot. The reason for this is that any small errors in the twice-differentiated series (acceleration) due to numerical inaccuracies etc. get compounded when integrating. When integrating twice, the error increases proportional to time squared.

This method will store all intermediate results. To prevent that we can wrap these functions with `Combine`

In [ ]:
pos_to_acc_to_pos = f.Combine([pos_to_vel, vel_to_acc, acc_to_vel, vel_to_pos], name="mouse_pos_2")

system = System(sampler, [pos_to_acc_to_pos])
system.start()

In [ ]:
from genki_signals.frontends import Line, WidgetDashboard

pos =  Line(system, "timestamp", "mouse_pos")
pos2 = Line(system, "timestamp", "mouse_pos_2")

frontend = WidgetDashboard([pos, pos2])
frontend

In [ ]:
system.stop()

Lastly let's create a custom `SignalFunction`. Our function will be an image filter, implemented by convolving some kernel with an input (image) signal. It will also turn the images into grayscale.

To do that we need to extend the `SignalFunction` base class. We technically only need to implement a `__call__` method for our class, but we will also override the default `__init__` method:

In [ ]:
import numpy as np
from genki_signals.functions import SignalFunction
from scipy import ndimage

class ConvGrayscaleImage(SignalFunction):
    def __init__(self, input_signal, name, kernel, inverse=False):
        super().__init__(input_signal, name=name, params={"kernel": kernel, "inverse": inverse})
        self.kernel = np.array(kernel) / np.linalg.norm(kernel)
        self.inverse = inverse

    def __call__(self, signal):
        grayscale = np.average(signal, axis=0, weights=[0.2989, 0.5870, 0.1140])
        l = [ndimage.convolve(grayscale[..., i], self.kernel) for i in range(grayscale.shape[-1])]
        if self.inverse:
            return 255 - np.stack(l, axis=-1)
        return np.stack(l, axis=-1)

The `__call__` method will receive an image array of shape `(n_channels, height, width, time)`. This shape depends on the input signal we define, but the main rule is that the last dimension is reserved for time. We start by making the image grayscale by averaging over the `n_channels` dimension. We then compute the convolved images, one per sample (`time` dimension). The `inverse` parameter allows us to also invert the colors of the image. Finally, we recreate the `time` axis using `np.stack()`

In the `__init__` method we call `super().__init__()` with some arguments, this needs to be done in a specific way. We will explain this in a bit, but first let's try our function in action:

In [ ]:
from scipy import signal

def gaussian_kernel(n, std):
    """
    Generates an n x n matrix with a centered gaussian 
    of standard deviation std centered on it.
    """
    gaussian1D = signal.gaussian(n, std)
    gaussian2D = np.outer(gaussian1D, gaussian1D)
    return gaussian2D

kernel = np.array([[-1, -1, -1, -1, -1],
                   [-1,  1,  2,  1, -1],
                   [-1,  2,  4,  2, -1],
                   [-1,  1,  2,  1, -1],
                   [-1, -1, -1, -1, -1]])

gaussian = gaussian_kernel(5, 1)
gaussian_edges = ndimage.convolve(gaussian, kernel)

conv = ConvGrayscaleImage("video", "video_edges", gaussian_edges, True)

In [ ]:
video = CameraSource(0)
sampler = Sampler({"video": video}, sample_rate=60)
system = System(sampler, [conv])

system.start()

In [ ]:
from genki_signals.frontends import Video, WidgetDashboard

video_normal = Video(system, "video")
video_edges = Video(system, "video_edges")

frontend = WidgetDashboard([video_normal, video_edges])

frontend

In [ ]:
system.stop()

All signal functions can be serialized into JSON. This is very useful for reasons we will explain in the next notebook, but it does have some nuances. This is the reason for the particular call to `__init__` we saw earlier:

        super().__init__(input_signal, name=name, params={"kernel": kernel, "inverse": inverse})

Each signal function must be initialized with these arguments in this order. The name of its input signal or signals, followed by a name given to the resulting signal, and finally a dict of extra parameters required to make the signal work. These three things, along with the class name, should uniquely specify a signal function and enable us to serialize and deserialize them. 

For example, the serialized JSON for our `conv` function is:

In [ ]:
import json
from genki_signals.functions.serialization import encode_signal_fn

print(json.dumps(conv, default=encode_signal_fn, indent=4))

This is enough to recreate the exact same function (provided the underlying code doesn't change) and thus persist a collection of such functions to disk. 

The next notebook will dive a bit deeper into the `System` class and data recording, and how this serialization property of signal functions can be useful.